## Notebook to retrieve IWP from GMI TB and other auxiliary data using QRNN

In [30]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from torch.utils.data import DataLoader, random_split
from iwc2tb.GMI.gmiData2 import gmiData
import os

from typhon.retrieval.qrnn import set_backend, QRNN
set_backend("pytorch")
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### set hyperparameters

In [2]:
#time.sleep(60 * 60)
#quantiles         = np.array([0.002, 0.03, 0.10, 0.16, 0.25, 0.34, 0.5, 0.66, 0.75, 0.84, 0.90, 0.97, 0.998])

quantiles         = np.arange(0.05, 1, 0.05)
quantiles         = np.linspace(0.01, 0.99, 128)
print(quantiles)
batchSize         = 256

depth             = 5
width             = 256 # 512
convergence_epoch = 5
maximum_epoch     = 80

inputs            = np.array(["ta", "t2m",  "wvp","lat", "z0", "stype"])
#inputs            = np.array(["ta", "t2m",  "wvp", "lat"])
#inputs            = np.array(["ta", "t2m", "wvp", "lat", "stype"])
ninputs           = len(inputs) + 3 + 10

outputs           = "iwp"

inChannels        = np.array(['166.5V', '166.5H', '183+-7', '183+-3'], dtype=object)


#latlims           = [45, 65] 
latlims           = [0, 65]
xlog              = True

outputfile        = os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_loglinear_jul.nc")

[0.01       0.01771654 0.02543307 0.03314961 0.04086614 0.04858268
 0.05629921 0.06401575 0.07173228 0.07944882 0.08716535 0.09488189
 0.10259843 0.11031496 0.1180315  0.12574803 0.13346457 0.1411811
 0.14889764 0.15661417 0.16433071 0.17204724 0.17976378 0.18748031
 0.19519685 0.20291339 0.21062992 0.21834646 0.22606299 0.23377953
 0.24149606 0.2492126  0.25692913 0.26464567 0.2723622  0.28007874
 0.28779528 0.29551181 0.30322835 0.31094488 0.31866142 0.32637795
 0.33409449 0.34181102 0.34952756 0.35724409 0.36496063 0.37267717
 0.3803937  0.38811024 0.39582677 0.40354331 0.41125984 0.41897638
 0.42669291 0.43440945 0.44212598 0.44984252 0.45755906 0.46527559
 0.47299213 0.48070866 0.4884252  0.49614173 0.50385827 0.5115748
 0.51929134 0.52700787 0.53472441 0.54244094 0.55015748 0.55787402
 0.56559055 0.57330709 0.58102362 0.58874016 0.59645669 0.60417323
 0.61188976 0.6196063  0.62732283 0.63503937 0.64275591 0.65047244
 0.65818898 0.66590551 0.67362205 0.68133858 0.68905512 0.696771

## training

In [3]:
def train(depth, width, batchSize, convergence_epoch, maximum_epoch, training_data, validation_data):
        qrnn = QRNN(ninputs, quantiles, (depth, width , "relu"))
        for lr in [  0.01, 0.001, 0.0001]:
            print ("NEW LEARNING RATE")
            results = qrnn.train(
                training_data,
                validation_data,
                batch_size=batchSize,
                momentum = 0.9,
                sigma_noise=None,
                initial_learning_rate= lr ,
                maximum_epochs=maximum_epoch,
                convergence_epochs= convergence_epoch,    
                gpu=True)

        return results, qrnn



In [4]:
ninputs

19

### read training data

In [31]:
data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train_july.nc"), 
               inputs, outputs, latlims = latlims,
               batch_size = batchSize, log_iwp = xlog)  

n = len(data)
n_train = int(0.9 * n)
n_val = n - n_train

training_data, validation_data = random_split(data, [n_train, n_val])
results = []

print(len(validation_data))

1199


In [35]:
data[0]

(tensor([[-1.0896, -1.4544, -0.5910,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.5000,  0.4877,  0.5907,  ...,  0.0000,  0.0000,  0.0000],
         [-0.2976, -0.1426, -0.8263,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-1.1852, -2.0463, -0.6241,  ...,  0.0000,  0.0000,  0.0000],
         [-2.8628, -2.3462, -1.5342,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.2102,  0.3472,  0.1735,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([-5.5278e+00, -1.2937e+01, -3.5534e+00, -6.5305e+00, -9.9240e+00,
         -3.3283e-01, -9.6194e+00, -1.0469e+01, -4.2973e+00, -5.4722e+00,
         -1.1773e+01, -1.0880e+01, -1.1715e+01, -1.3364e+01, -1.2830e+01,
         -4.7642e+00, -1.1970e+01, -1.3305e+01, -1.1648e+01, -1.3451e+01,
         -4.0981e+00, -9.3767e+00, -4.2614e+00, -1.2766e+01, -1.2209e+01,
         -1.2224e+01, -9.3868e+00, -7.0911e+00, -3.7858e+00, -1.0824e+01,
         -1.0343e+01, -2.7960e+00, -9.0953e+00, -8.4413e+00, -9.8310e+00,
         -2.4685e+00, -9.4639e+00, -6.22

In [34]:
validation_data[0]

(tensor([[ 0.2854,  0.4188,  0.1763,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.9574,  0.6422,  1.0999,  ...,  0.0000,  0.0000,  0.0000],
         [-0.3253, -0.3179,  0.0297,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.2461,  0.2006,  0.6121,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.3818,  0.5743,  0.0180,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0092,  0.8689,  0.4717,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([-11.4698, -10.5683,  -4.1316,  -3.9861,  -1.7312, -13.7560,   0.8510,
         -12.6312,   1.8501,  -0.7669, -13.4940, -13.3637,  -5.2691, -12.7045,
          -2.6064, -13.7676, -11.7544, -13.2681, -12.7416, -10.9272,  -0.3790,
         -10.1253, -12.8256, -12.6878,  -1.0776,  -7.7267, -10.0026,  -9.6250,
         -10.1759,  -2.9466,  -9.2969, -11.8083, -11.3286,  -9.2461, -11.6550,
         -13.2892, -12.5927, -13.1012, -11.0085,  -7.4686, -12.6401, -13.2826,
          -1.7691, -13.0055,  -2.7258,  -4.4967, -13.0692,  -5.2941,  -6.7907,
     

### start training

In [6]:
results, qrnn = train(depth, width, batchSize, convergence_epoch, maximum_epoch, training_data, validation_data)


NEW LEARNING RATE
2153854.784795463 2760957 / 10785, Training error: 0.780
Epoch 0 / 80: Training error: 0.780, Validation error: 0.658, Learning rate: 0.01000
1799311.583863318 2760957 / 10785, Training error: 0.652
Epoch 1 / 80: Training error: 0.652, Validation error: 0.646, Learning rate: 0.01000
1776185.251058519 2760957 / 10785, Training error: 0.643
Epoch 2 / 80: Training error: 0.643, Validation error: 0.639, Learning rate: 0.01000
1763396.8032799363 2760957/ 10785, Training error: 0.639
Epoch 3 / 80: Training error: 0.639, Validation error: 0.634, Learning rate: 0.01000
1757353.989262879 2760957 / 10785, Training error: 0.637
Epoch 4 / 80: Training error: 0.637, Validation error: 0.634, Learning rate: 0.01000
1751402.1849919558 2760957/ 10785, Training error: 0.634
Epoch 5 / 80: Training error: 0.634, Validation error: 0.637, Learning rate: 0.01000
Epoch 6 / 80: Batch 938 / 10785, Training error: 0.634

KeyboardInterrupt: 

In [ ]:
qrnn.save(os.path.expanduser(outputfile))

#qrnn = QRNN.load(os.path.expanduser('~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_iwp5.nc'))
fig, ax = plt.subplots(1, 1)
ax.plot(results['training_errors'])
ax.plot(results['validation_errors'])

In [ ]:
x, y = qrnn.calibration(validation_data)
f, ax = plt.subplots(1, 1, figsize = [8, 8])
ax.plot(x, y, marker = "o", c = 'r')
ax.plot(x, x, ls = ":", c = "k")
#ax.set_xlim([0.1, 0.9])
#ax.set_ylim([0.1, 0.9])
ax.set_aspect(1.0)
ax.set_xlabel("Predicted frequency")
ax.set_ylabel("Observed frequency")
f.savefig("calibration.png", bbox_inches = "tight")

In [ ]:
imedian            = np.argwhere((quantiles >= 0.50) & (quantiles < 0.51))[0][0]
y_pre = []
y = []
y_prior = []
y_pos_mean = []
x_in = []

nbatch = len(validation_data)
print (nbatch)
for i in range(nbatch):
    
    xx, yy = validation_data[i]
    
    x = xx.detach().numpy() 

    y_pre.append(qrnn.predict(x)) 
    y_pos_mean.append((qrnn.posterior_mean(x)))
       
    y.append(yy.detach().numpy())
    x_in.append(x)

x_in = np.concatenate(x_in, axis = 0)
y_pre = np.concatenate(y_pre, axis = 0)
y = np.concatenate(y, axis= 0)
y_pos_mean = np.concatenate(y_pos_mean, axis = 0)


y_pre = np.where(y_pre > 0, y_pre + 1.0, np.exp(y_pre))
y = np.where(y > 0, y + 1.0, np.exp(y))

plt.rcParams.update({'font.size': 20})
bins1 = np.arange(0, 300, 0.1)
fig, ax = plt.subplots(1, 1, figsize = [8, 8])
ax.hist(y_pre[:, imedian], bins1, density = True , histtype = "step", label = "predicted")


ax.hist(y, bins1, density = True, histtype = "step", label = "observed")
ax.set_yscale('log')
ax.set_xscale('log')

ax.legend()
ax.set_ylabel("PDF")
ax.set_xlabel("IWP[kg/m2]")

In [ ]:
y_pre.max()

In [ ]:
import scipy
from matplotlib import ticker, cm
xyrange = [[0, 25], [0, 25]] # data range


bins = [50, 50] # number of bins
hh, locx, locy = np.histogram2d(y, y_pos_mean, 
                                range=xyrange, bins=bins, density = True)
posx = np.digitize(y, locx)
posy = np.digitize(y_pos_mean, locy)

fig, ax = plt.subplots(1, 1, figsize = [10, 8])
cs = ax.contourf(np.flipud(hh.T),
                extent=np.array(xyrange).flatten(), 
            locator= ticker.LogLocator(), origin='upper')
cbar = fig.colorbar(cs)
ax.set_ylim([0, 20])
ax.set_xlim([0, 20])
xy = np.arange(0, 13, 1)
yy = xy
ax.plot(xy, yy)
ax.set_ylabel("IWP Predicted [kg/m2]")
ax.set_xlabel("IWP Observed [kg/m2]")
ax.grid('on')
#ax.set_yscale('log')
#ax.set_xscale('log')

In [ ]:
bins = np.array([0.0,.0001,.00025,.0005, 0.001,.0025,.005,
                 0.0075, 0.01, 0.025, 0.05, 0.075, .1, .25,
                 .5, .75, 1, 2, 3, 4, 5, 6, 8, 10, 12, 16, 20, 24, 28, 32, 50])

#bins = np.logspace(np.log10(0.001), np.log10(25), 500)
iy = np.digitize((y), bins)
iyp = np.digitize((y_pre[:, imedian]), bins)

iby = np.bincount(iy, minlength = len(bins), weights = y)
ibyp = np.bincount(iyp, minlength = len(bins), weights = y_pos_mean)

niby = np.bincount(iy, minlength = len(bins))
nibyp = np.bincount(iyp, minlength = len(bins))

fig, ax = plt.subplots(1, 1, figsize = [5, 5])
ax.plot(iby/niby, ibyp/nibyp, 'o-', linewidth = 2.5)

ax.set_yscale("log")
ax.set_xscale("log")
ax.grid("on")
xx = bins
yy = bins

ax.plot(xx, yy, 'k--')

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
y_pre.max()